# Scraper (Host Uni + Mappable Mods)

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [10]:
import os
from dotenv import load_dotenv
from supabase import create_client, Client

SUPABASE_URL = os.environ.get("SUPABASE_URL")
SUPABASE_KEY = os.environ.get("SUPABASE_KEY")

if not SUPABASE_URL or not SUPABASE_KEY:
    raise ValueError("SUPABASE_URL and SUPABASE_KEY must be set in .env")

supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

# Insert CS Exchange Mods

In [ ]:
from bs4 import BeautifulSoup                                                                                                                                                                                                                                                 
# First, get all valid module codes from Supabase                                                                                    ─
result = supabase.table("modules").select("module_code").execute()                                                                      
valid_modules = {row["module_code"] for row in result.data}                                                                             
print(f"Found {len(valid_modules)} valid NUS modules")                                                                                  
                                                                                                                                        
# Read local HTML                                                                                                                    
with open("cs_courses.html", encoding="utf-8") as f:                                                                                    
    soup = BeautifulSoup(f, "html.parser")                                                                                              
                                                                                                                                        
# Select table                                                                                                                          
table = soup.find("table", id="N_EXSP_DRVD$scroll$0")                                                                                   
rows = table.select("tr[id^='trN_EXSP_DRVD']")                                                                                          
                                                                                                                                        
# Stream rows in batches                                                                                                                
BATCH_SIZE = 500                                                                                                                        
batch = []                                                                                                                              
skipped = 0                                                                                                                             
                                                                                                                                        
for i, row in enumerate(rows, start=1):                                                                                                 
    cells = row.find_all("td")                                                                                                          
                                                                                                                                        
    nus_course = cells[8].get_text(strip=True)                                                                                          
                                                                                                                                        
    # Skip if NUS course doesn't exist in modules table                                                                              
    if nus_course and nus_course not in valid_modules:                                                                                  
        skipped += 1                                                                                                                    
        continue                                                                                                                        
                                                                                                                                        
    data = {                                                                                                                            
        "faculty": cells[0].get_text(strip=True),                                                                                       
        "partner_univ": cells[1].get_text(strip=True),                                                                                  
        "pu_course": cells[2].get_text(strip=True),                                                                                     
        "pu_course_title": cells[3].get_text(strip=True),                                                                               
        "pu_course_units": float(cells[4].get_text(strip=True)) if cells[4].get_text(strip=True) else None,                             
        "nus_course": nus_course if nus_course else None,                                                                               
        "nus_course_title": cells[9].get_text(strip=True),                                                                              
        "nus_course_units": float(cells[10].get_text(strip=True)) if cells[10].get_text(strip=True) else None,                          
        "preapproved": True if cells[14].find("input", {"value": "Y"}) else False                                                       
    }                                                                                                                                   
                                                                                                                                        
    batch.append(data)                                                                                                                  
                                                                                                                                        
    if len(batch) >= BATCH_SIZE:                                                                                                        
        supabase.table("exchange_modules").insert(batch).execute()                                                                      
        print(f"Inserted {len(batch)} rows")                                                                                            
        batch.clear()                                                                                                                   
                                                                                                                                        
# Insert remaining                                                                                                                      
if batch:                                                                                                                               
    supabase.table("exchange_modules").insert(batch).execute()                                                                          
    print(f"Inserted final {len(batch)} rows")                                                                                          
                                                                                                                                        
print(f"✅ Done! Skipped {skipped} rows with invalid NUS course codes")

Found 7067 valid NUS modules
Inserted 500 rows
Inserted 500 rows
Inserted 500 rows
Inserted 500 rows
Inserted 500 rows
Inserted 500 rows
Inserted 500 rows
Inserted 500 rows
Inserted 500 rows
Inserted 500 rows
Inserted final 1 rows
✅ Done! Skipped 301 rows with invalid NUS course codes


# Insert Business Exchange Module 

In [19]:
# First, get all valid module codes from Supabase                                                                                    ─
result = supabase.table("modules").select("module_code").execute()                                                                      
valid_modules = {row["module_code"] for row in result.data}                                                                             
print(f"Found {len(valid_modules)} valid NUS modules")                                                                                  
                                                                                                                                        
# Read local HTML                                                                                                                    
with open("business_courses.html", encoding="utf-8") as f:                                                                                    
    soup = BeautifulSoup(f, "html.parser")                                                                                              
                                                                                                                                        
# Select table                                                                                                                          
table = soup.find("table", id="N_EXSP_DRVD$scroll$0")                                                                                   
rows = table.select("tr[id^='trN_EXSP_DRVD']")                                                                                          
                                                                                                                                        
# Stream rows in batches                                                                                                                
BATCH_SIZE = 500                                                                                                                        
batch = []                                                                                                                              
skipped = 0                                                                                                                             
                                                                                                                                        
for i, row in enumerate(rows, start=1):                                                                                                 
    cells = row.find_all("td")                                                                                                          
                                                                                                                                        
    nus_course = cells[8].get_text(strip=True)                                                                                          
                                                                                                                                        
    # Skip if NUS course doesn't exist in modules table                                                                              
    if nus_course and nus_course not in valid_modules:                                                                                  
        skipped += 1                                                                                                                    
        continue                                                                                                                        
                                                                                                                                        
    data = {                                                                                                                            
        "faculty": cells[0].get_text(strip=True),                                                                                       
        "partner_univ": cells[1].get_text(strip=True),                                                                                  
        "pu_course": cells[2].get_text(strip=True),                                                                                     
        "pu_course_title": cells[3].get_text(strip=True),                                                                               
        "pu_course_units": float(cells[4].get_text(strip=True)) if cells[4].get_text(strip=True) else None,                             
        "nus_course": nus_course if nus_course else None,                                                                               
        "nus_course_title": cells[9].get_text(strip=True),                                                                              
        "nus_course_units": float(cells[10].get_text(strip=True)) if cells[10].get_text(strip=True) else None,                          
        "preapproved": True if cells[14].find("input", {"value": "Y"}) else False                                                       
    }                                                                                                                                   
                                                                                                                                        
    batch.append(data)                                                                                                                  
                                                                                                                                        
    if len(batch) >= BATCH_SIZE:                                                                                                        
        supabase.table("exchange_modules").insert(batch).execute()                                                                      
        print(f"Inserted {len(batch)} rows")                                                                                            
        batch.clear()                                                                                                                   
                                                                                                                                        
# Insert remaining                                                                                                                      
if batch:                                                                                                                               
    supabase.table("exchange_modules").insert(batch).execute()                                                                          
    print(f"Inserted final {len(batch)} rows")                                                                                          
                                                                                                                                        
print(f"✅ Done! Skipped {skipped} rows with invalid NUS course codes")

Found 7067 valid NUS modules
Inserted 500 rows
Inserted 500 rows
Inserted final 259 rows
✅ Done! Skipped 1327 rows with invalid NUS course codes
